# Aggregation of Oceanic Nino Index(ONI) and Monthly Global Precipitation Means

In [9]:
# Initialize notebook environment.
%matplotlib inline
import logging
import boto3
from botocore.exceptions import ClientError
import botocore
import datetime
import matplotlib.pyplot as plt
import os.path
import xarray as xr
import json
import numpy as np
import pandas as pd 
import requests

### Oceanic Nino Index DataFrame

In [8]:
rainfall_avgs = json.load(open('rainfall_avgs.json', 'r'))
oni = pd.read_csv('https://query.data.world/s/72sjhekyakcsiehjtd7yjuhccedtae')
oni

,Year,DJF,JFM,FMA,MAM,AMJ,MJJ,JJA,JAS,ASO,SON,OND,NDJ
0,1950,-1.4,-1.2,-1.1,-1.2,-1.1,-0.9,-0.6,-0.6,-0.5,-0.6,-0.7,-0.8
1,1951,-0.8,-0.6,-0.2,0.2,0.2,0.4,0.5,0.7,0.8,0.9,0.7,0.6
2,1952,0.5,0.4,0.4,0.4,0.4,0.2,0.0,0.1,0.2,0.2,0.2,0.3
3,1953,0.5,0.6,0.7,0.7,0.7,0.7,0.7,0.7,0.8,0.8,0.8,0.7
4,1954,0.7,0.4,0.0,-0.4,-0.5,-0.5,-0.5,-0.7,-0.7,-0.6,-0.5,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2013,-0.4,-0.4,-0.3,-0.2,-0.2,-0.2,-0.3,-0.3,-0.2,-0.3,-0.3,-0.3
64,2014,-0.5,-0.5,-0.4,-0.2,-0.1,0.0,-0.1,0.0,0.1,0.4,0.5,0.6
65,2015,0.6,0.5,0.6,0.7,0.8,1.0,1.2,1.4,1.7,2.0,2.2,2.3
66,2016,2.2,2.0,1.6,1.1,0.6,0.1,-0.3,-0.6,-0.8,-0.8,-0.8,-0.7


### Defining DataFrames for Each Country 

In [3]:
country_dataframes = {
    'argentina': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'guatemala': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'peru': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'botswana': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'zimbabwe': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'mozambique': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'south_africa': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'el_salvador': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'honduras': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'nicaragua': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'haiti': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'se_brazil': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'ethiopia': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'kenya': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'somalia': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']),
    'malawi': pd.DataFrame(columns=['djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj'])
}

### Switch Function for Aggregating Means

In [4]:
def one(df, mean, year):
    df.loc[year,'ndj'] = mean 
    df.loc[year,'djf'] = mean
    df.loc[year,'jfm'] = mean
    
def two(df, mean, year):
    df.loc[year,'djf'] += mean 
    df.loc[year,'jfm'] += mean
    df.loc[year,'fma'] = mean
    
def three(df, mean, year):
    df.loc[year,'jfm'] += mean
    df.loc[year,'fma'] += mean
    df.loc[year,'mam'] = mean
    df.loc[year,'jfm'] /= 3
    
def four(df, mean, year):
    df.loc[year,'fma'] += mean
    df.loc[year,'mam'] += mean
    df.loc[year,'amj'] = mean
    df.loc[year,'fma'] /= 3
    
def five(df, mean, year):
    df.loc[year,'mam'] += mean
    df.loc[year,'amj'] += mean
    df.loc[year,'mjj'] = mean
    df.loc[year,'mam'] /= 3
    
def six(df, mean, year):
    df.loc[year,'amj'] += mean
    df.loc[year,'mjj'] += mean
    df.loc[year,'jja'] = mean
    df.loc[year,'amj'] /= 3
    
def seven(df, mean, year):
    df.loc[year,'mjj'] += mean
    df.loc[year,'jja'] += mean
    df.loc[year,'jas'] = mean
    df.loc[year,'mjj'] /= 3
    
def eight(df, mean, year):
    df.loc[year,'jja'] += mean
    df.loc[year,'jas'] += mean
    df.loc[year,'aso'] = mean
    df.loc[year,'jja'] /= 3
    
def nine(df, mean, year):
    df.loc[year,'jas'] += mean
    df.loc[year,'aso'] += mean
    df.loc[year,'son'] = mean
    df.loc[year,'jas'] /= 3
    
def ten(df, mean, year):
    df.loc[year,'aso'] += mean
    df.loc[year,'son'] += mean
    df.loc[year,'ond'] = mean
    df.loc[year,'aso'] /= 3
    
def eleven(df, mean, year):
    df.loc[year,'son'] += mean
    df.loc[year,'ond'] += mean
    df.loc[year,'ndj'] += mean
    df.loc[year,'son'] /= 3
    
def twelve(df, mean, year):
    df.loc[year,'ond'] += mean
    df.loc[year,'ndj'] += mean
    df.loc[year,'djf'] += mean
    df.loc[year,'ond'] /= 3
    df.loc[year,'ndj'] /= 3
    df.loc[year,'djf'] /= 3
 
def aggregate_means(df, mean, month, year):
    switcher = {
        1: one,
        2: two,
        3: three,
        4: four,
        5: five,
        6: six,
        7: seven,
        8: eight,
        9: nine,
        10: ten,
        11: eleven,
        12: twelve
    }
    # Get the function from switcher dictionary
    func = switcher[int(month)]
    # Execute the function
    func(df, float(mean), int(year))

In [5]:
for year in rainfall_avgs:
    curr_year = rainfall_avgs[year]
    for month in curr_year:
        curr_month = curr_year[month]
        for risk in curr_month:
            locations = curr_month[risk]
            for loc in locations:
                country = loc
                month = month
                year = year
                avg_rainfall = locations[loc]
                country_df = country_dataframes[loc]
                aggregate_means(country_df, avg_rainfall, month, year)

### First 5 Rows of the Argentina Three Month Mean DataFrame

In [7]:
country_dataframes['argentina'].head()

,djf,jfm,fma,mam,amj,mjj,jja,jas,aso,son,ond,ndj
1979,1.11987,1.08862,1.52862,2.48561,3.60931,4.50846,4.47233,3.63779,3.00693,2.92832,2.67041,1.96212
1980,1.72387,1.29613,1.26117,1.03437,1.59124,1.67601,1.81859,1.52292,1.87741,2.56143,2.72796,2.32367
1981,2.40827,1.89265,1.92263,1.87861,3.31706,3.49508,4.46286,3.48503,3.99089,3.36309,3.4181,2.70423
1982,1.51444,1.21561,1.71483,2.05879,2.58304,2.81429,3.33382,3.43171,3.08405,3.1945,2.87119,2.48838
1983,2.40562,1.99449,2.04241,2.75397,2.80505,3.69756,3.61357,4.6792,4.59814,4.42813,3.72256,2.86318
